In [1]:
import os
os.environ['LANGCHAIN_TRACING_V2'] = 'true'
os.environ['LANGCHAIN_ENDPOINT'] = 'https://api.smith.langchain.com'
os.environ['LANGCHAIN_API_KEY'] = 'ls__e4fe4b0966434a62a5590fd260a248df'
os.environ['OPENAI_API_KEY'] = 'sk-proj-65rh8ytSwOKvHI0BQ9t5T3BlbkFJ50gQcbM2uHngWXtjR9Rl'

In [2]:
import bs4
from langchain import hub
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.document_loaders import WebBaseLoader
from langchain_community.document_loaders import UnstructuredHTMLLoader
from langchain_community.document_loaders import BSHTMLLoader
from langchain_text_splitters import HTMLHeaderTextSplitter
from langchain_community.vectorstores import Chroma
from langchain_community.embeddings.sentence_transformer import (SentenceTransformerEmbeddings,)


In [3]:
with open('Links.txt' , 'r') as f:
    lines = f.readlines()

lines = [line.strip() for line in lines]

# Load Documents
loader = WebBaseLoader(lines)
docs = loader.load()

In [4]:
# Split
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
splits = text_splitter.split_documents(docs)

In [5]:
# create the open-source embedding function
embedding_function = SentenceTransformerEmbeddings(model_name="all-MiniLM-L6-v2")

# Embed
vectorstore = Chroma.from_documents(documents=splits, embedding=embedding_function)


In [6]:
# create the open-source embedding function
embedding_function = SentenceTransformerEmbeddings(model_name="all-MiniLM-L6-v2")

# Embed
vectorstore = Chroma.from_documents(documents=splits, embedding=embedding_function)

# query it
questions = ["What are the statutory working hours in Poland?",
             "How many days of paid vacation am I allowed to take?",
             "Can I work overtime and how much do I get paid for it?",
             "What is the minimum wage in Poland?",
             "What are my rights with regards to taking a sick leave?",
             "What legal provisions govern maternity and paternity leave in Poland?",
             "How does Polish labor law protect employees against workplace discrimination?",
             "What are the rules regarding termination of employment contracts in Poland?",
             "What obligations do employers have in terms of workplace safety according to Polish labor regulations?",
             "How does Polish law regulate the employment of minors?",
             "What are the regulations concerning probationary periods for new employees in Poland?",
             "How does Polish labor law address the issue of employee representation and collective bargaining?",
             "What are the requirements for employers regarding employee documentation and record-keeping in Poland?",
             "What measures does Polish labor law provide for in cases of workplace harassment or bullying?",
             "How does Polish law address the issue of non-compete clauses in employment contracts?",
             "What rights do employees have regarding parental leave in Poland?",
             "How does Polish labor law regulate the employment of foreign nationals?",
             "What are the rules regarding working conditions, such as breaks and rest periods, in Poland?",
             "What is the procedure for resolving labor disputes in Poland?",
             "How does Polish labor law address the issue of employee privacy rights in the workplace?",
             "What are the regulations concerning severance pay for terminated employees in Poland?",
             "How does Polish law regulate the outsourcing of labor or subcontracting arrangements?",
             "What obligations do employers have in terms of providing employee benefits, such as healthcare and pension contributions?",
             "How does Polish labor law address the issue of equal pay for equal work?",
             "What are the rules regarding employee entitlements to notice periods upon termination of employment?",
             "How does Polish law regulate the use of fixed-term contracts or temporary employment arrangements?",
             "What measures are in place to prevent workplace accidents and ensure occupational health and safety in Poland?",
             "What rights do employees have regarding whistleblowing and reporting misconduct in the workplace?",
             "How does Polish labor law address the issue of employee training and development opportunities?",
             "What are the regulations concerning the transfer of employees during company mergers or acquisitions in Poland?",]

related_source_links = {}
related_chunk = {}

for question in questions:
    related_doc = vectorstore.similarity_search(question, k=1)
    related_source_links[question] = related_doc[0].metadata['source']
    related_chunk[question] = related_doc[0].dict()['page_content']

In [7]:
related_chunk

{'What are the statutory working hours in Poland?': 'Change language\nEN\nBack\n\n\nPL - polski\n\n\nClose the menu\n\n\n\n\n\n\n\n\n\nMinistry of Family, Labour and Social Policy\n\n\nWhat we do\n\n\nLabour law\n\n\nBasic information on labour law\n\n\nWorking time setting and settlement\n\n\nWorking time setting \n\n\nStandards and quantities of working time\n\n\n\nBack\n\n\n\nStandards and quantities of working time\n\nThe provisions of the Labour Code specify working time standards, that is: 8\xa0hours a day and an average of 40\xa0hours per week in an average 5‑day working week in the adopted reference period. On the basis of these standards, the employer determines the working time, that is the number of hours and days to work in the adopted working time reference period.\nThe number of these hours is calculated as follows:\n\nby multiplying 40\xa0hours by the number of weeks per reference period, then\nby adding – to the number of hours obtained – the product of 8\xa0hours and t

In [22]:
def clean_response(response):
    # Remove unwanted characters and clean up the response
    response = response.replace("\n", " ").replace("\xa0", " ").strip()
    
    # Replace multiple spaces with a single space
    while "  " in response:
        response = response.replace("  ", " ")
    
    # Add line breaks for better readability if needed
    # For example, breaking long paragraphs into multiple lines
    max_length = 120
    words = response.split(' ')
    cleaned_response = ''
    current_length = 0
    
    for word in words:
        if current_length + len(word) + 1 > max_length:
            cleaned_response += '\n'
            current_length = 0
        cleaned_response += word + ' '
        current_length += len(word) + 1
    
    return cleaned_response.strip()

def clean_responses(responses_dict):
    cleaned_responses = {}
    for question, response in responses_dict.items():
        cleaned_responses[question] = clean_response(response)
    return cleaned_responses

related_chunk = clean_responses(related_chunk)

In [23]:
related_chunk

{'What are the statutory working hours in Poland?': 'Change language EN Back PL - polski Close the menu Ministry of Family, Labour and Social Policy What we do Labour law \nBasic information on labour law Working time setting and settlement Working time setting Standards and quantities of \nworking time Back Standards and quantities of working time The provisions of the Labour Code specify working time \nstandards, that is: 8 hours a day and an average of 40 hours per week in an average 5‑day working week in the adopted \nreference period. On the basis of these standards, the employer determines the working time, that is the number of \nhours and days to work in the adopted working time reference period. The number of these hours is calculated as \nfollows: by multiplying 40 hours by the number of weeks per reference period, then by adding – to the number of hours \nobtained – the product of 8 hours and the number of days remaining until the end of the reference period, from Monday \nt

In [19]:
gold_response = {
    "What are the statutory working hours in Poland?": "The standard working hours in Poland are 8 hours per day and 40 hours per week, spread over 5 days. Overtime is limited to 150 hours per year unless agreed otherwise, and employees must receive at least 11 hours of rest per day and 35 hours of rest per week.",
    
    "How many days of paid vacation am I allowed to take?": "Employees in Poland are entitled to either 20 or 26 days of paid vacation per year, depending on their length of service. Employees with less than 10 years of service are entitled to 20 days, while those with 10 or more years of service are entitled to 26 days.",
    
    "Can I work overtime and how much do I get paid for it?": "Yes, you can work overtime. Overtime is compensated at 150% of the regular rate for overtime on a regular day and 200% for overtime on Sundays, holidays, and night work. Alternatively, employees can receive additional time off instead of higher pay.",
    
    "What is the minimum wage in Poland?": "As of 2024, the minimum wage in Poland is 4,242 PLN per month and 27.70 PLN per hour. These rates are subject to annual adjustments.",
    
    "What are my rights with regards to taking a sick leave?": "Employees in Poland are entitled to sick leave and receive sick pay. The first 33 days of sick leave are paid by the employer at 80% of the employee's salary, and from the 34th day onwards, it is paid by the Social Insurance Institution (ZUS). The sick pay rate can increase to 100% in cases of illness due to work accidents or during pregnancy.",
    
    "What legal provisions govern maternity and paternity leave in Poland?": "Maternity leave in Poland is 20 weeks for a single birth, with the possibility of additional leave for multiple births. Fathers are entitled to 2 weeks of paternity leave. Parents can also take parental leave of up to 32 weeks (34 weeks for multiple births), which can be shared between the parents.",
    
    "How does Polish labor law protect employees against workplace discrimination?": "Polish labor law prohibits discrimination in employment on various grounds, including gender, age, disability, race, religion, nationality, political beliefs, and trade union membership. Employers are required to ensure equal treatment in hiring, promotion, training, and working conditions.",
    
    "What are the rules regarding termination of employment contracts in Poland?": "Termination of employment contracts in Poland requires notice, which varies based on the employee's length of service: 2 weeks for less than 6 months, 1 month for 6 months to 3 years, and 3 months for more than 3 years. Termination must be justified with a valid reason, and specific procedures must be followed.",
    
    "What obligations do employers have in terms of workplace safety according to Polish labor regulations?": "Employers in Poland are obligated to ensure safe and healthy working conditions, provide necessary safety training, conduct regular risk assessments, and comply with occupational health and safety regulations. They must also provide personal protective equipment and maintain accident registers.",
    
    "How does Polish law regulate the employment of minors?": "Employment of minors (persons under 18) in Poland is subject to strict regulations. Minors aged 15-18 can be employed only for light work that does not harm their health or development and must have completed compulsory schooling. Employment contracts for minors require consent from their legal guardians and approval from the labor inspectorate.",
    
    "What are the regulations concerning probationary periods for new employees in Poland?": "Probationary periods in Poland can last up to 3 months. During this period, either party can terminate the employment contract with shorter notice periods: 3 days for a probationary period of up to 2 weeks, 1 week for a probationary period between 2 weeks and 3 months, and 2 weeks for a probationary period of 3 months.",
    
    "How does Polish labor law address the issue of employee representation and collective bargaining?": "Employees in Poland have the right to form and join trade unions, which represent their interests in negotiations with employers. Collective bargaining agreements can be negotiated at the company or sectoral level, and they govern various employment conditions, including wages, working hours, and other benefits.",
    
    "What are the requirements for employers regarding employee documentation and record-keeping in Poland?": "Employers in Poland must maintain accurate and up-to-date personnel records for each employee, including employment contracts, personal data, records of working time, payroll documentation, and any disciplinary actions. These records must be kept for specified periods, generally 10 years, and must be accessible for inspection by labor authorities.",
    
    "What measures does Polish labor law provide for in cases of workplace harassment or bullying?": "Polish labor law obligates employers to prevent workplace harassment and bullying. Employers must implement measures to protect employees from such behavior, provide channels for reporting incidents, and take appropriate actions to address and resolve complaints. Failure to address harassment can result in legal consequences for the employer.",
    
    "How does Polish law address the issue of non-compete clauses in employment contracts?": "Non-compete clauses in Poland are enforceable if they are reasonable in scope, duration, and geographic area. The agreement must be in writing, and the employer is required to pay compensation to the employee for the duration of the non-compete period, which must be at least 25% of the employee's previous salary.",
    
    "What rights do employees have regarding parental leave in Poland?": "Employees in Poland are entitled to parental leave of up to 32 weeks (34 weeks for multiple births), which can be shared between both parents. Parental leave can be taken in one continuous period or split into up to 4 parts. During this leave, employees receive an allowance funded by the Social Insurance Institution (ZUS).",
    
    "How does Polish labor law regulate the employment of foreign nationals?": "Foreign nationals need a work permit to be employed in Poland, unless they are from EU/EEA countries or Switzerland. The employer must obtain the permit on behalf of the employee, who must also have a legal right to stay in Poland. There are different types of work permits depending on the nature and duration of employment.",
    
    "What are the rules regarding working conditions, such as breaks and rest periods, in Poland?": "Employees working more than 6 hours a day are entitled to a 15-minute paid break. Employers may provide an additional unpaid break of up to 60 minutes. Employees are entitled to at least 11 consecutive hours of rest per day and 35 consecutive hours of rest per week, including Sundays.",
    
    "What is the procedure for resolving labor disputes in Poland?": "Labor disputes in Poland can be resolved through negotiations, mediation, or arbitration. If these methods fail, disputes can be taken to labor courts. Employees can also seek assistance from trade unions or labor inspectors. Labor courts handle cases related to unfair dismissal, wage disputes, and other employment-related issues.",
    
    "How does Polish labor law address the issue of employee privacy rights in the workplace?": "Polish labor law protects employee privacy rights, including personal data protection and confidentiality. Employers can monitor employees only to the extent necessary for business purposes and must inform employees about the monitoring methods used. Unauthorized surveillance or misuse of personal data is prohibited.",
    
    "What are the regulations concerning severance pay for terminated employees in Poland?": "Employees in Poland are entitled to severance pay if they are dismissed due to organizational reasons, such as redundancy or liquidation of the company. Severance pay is based on the length of service: 1 month's salary for up to 2 years of service, 2 months' salary for 2-8 years of service, and 3 months' salary for more than 8 years of service.",
    
    "How does Polish law regulate the outsourcing of labor or subcontracting arrangements?": "Outsourcing and subcontracting in Poland must comply with labor laws, including ensuring that subcontracted workers receive the same rights and protections as direct employees. Employers must verify that subcontractors adhere to legal employment standards, and they remain liable for any violations of labor rights by the subcontractors.",
    
    "What obligations do employers have in terms of providing employee benefits, such as healthcare and pension contributions?": "Employers in Poland are required to contribute to social security, which includes healthcare, pensions, disability, and accident insurance. These contributions are deducted from the employee's salary and supplemented by employer contributions. Employers may also offer additional benefits, such as private health insurance or pension plans, as part of their employment contracts.",
    
    "How does Polish labor law address the issue of equal pay for equal work?": "Polish labor law mandates equal pay for equal work, prohibiting wage discrimination based on gender, age, race, or other factors. Employers must ensure that employees performing the same or equivalent work receive equal pay, and any discrepancies must be justifiable based on objective criteria such as qualifications, experience, or performance.",
    
    "What are the rules regarding employee entitlements to notice periods upon termination of employment?": "Notice periods in Poland vary based on the length of service: 2 weeks for employees with less than 6 months of service, 1 month for employees with 6 months to 3 years of service, and 3 months for employees with more than 3 years of service. During the notice period, employees are entitled to their regular salary and benefits.",
    
    "How does Polish law regulate the use of fixed-term contracts or temporary employment arrangements?": "Fixed-term contracts in Poland are limited to a maximum duration of 33 months and can be renewed up to three times. After this period, the contract automatically converts to an indefinite-term contract. Temporary employment agencies must comply with specific regulations, ensuring that temporary workers receive the same rights and protections as permanent employees.",
    
    "What measures are in place to prevent workplace accidents and ensure occupational health and safety in Poland?": "Polish labor law requires employers to implement comprehensive occupational health and safety measures, including regular risk assessments, safety training, provision of personal protective equipment, and adherence to safety standards. Employers must also establish health and safety committees and report workplace accidents to relevant authorities.",
    
    "What rights do employees have regarding whistleblowing and reporting misconduct in the workplace?": "Employees in Poland are protected when reporting misconduct or illegal activities in the workplace. Employers are prohibited from retaliating against whistleblowers, and employees can report concerns anonymously. There are legal provisions to protect the identity and employment of whistleblowers, ensuring they are not subject to dismissal or other adverse actions.",
    
    "How does Polish labor law address the issue of employee training and development opportunities?": "Employers in Poland are encouraged to provide training and development opportunities to employees to enhance their skills and qualifications. Employers may be required to offer training in specific cases, such as when new technologies or procedures are introduced. Training costs are generally covered by the employer, and employees may be granted time off for training.",
    
    "What are the regulations concerning the transfer of employees during company mergers or acquisitions in Poland?": "In the event of a company merger or acquisition, Polish labor law protects employees' rights by ensuring that their existing employment contracts and conditions are transferred to the new employer. The new employer must honor all existing employment terms and conditions, and any changes must be negotiated with employee representatives or trade unions."
}


In [21]:
import nltk
from nltk.translate.bleu_score import sentence_bleu, SmoothingFunction

# Function to compute BLEU score
def compute_bleu(reference, hypothesis):
    """
    Compute BLEU score between a reference sentence and a hypothesis sentence.
    :param reference: The gold standard response (list of tokens)
    :param hypothesis: The LLM generated response (list of tokens)
    :return: BLEU score
    """
    # Using SmoothingFunction to handle cases with few words or edge cases
    smoothing_function = SmoothingFunction().method1
    return sentence_bleu([reference], hypothesis, smoothing_function=smoothing_function)

benchmark_data = []

for question in questions:
    benchmark_data.append({"query": question, 
                           "gold_response": gold_response[question],
                           "llm_response": related_chunk[question]})

# Tokenize responses
for data in benchmark_data:
    data["gold_response_tokens"] = nltk.word_tokenize(data["gold_response"])
    data["llm_response_tokens"] = nltk.word_tokenize(data["llm_response"])

# # Compute BLEU scores
for data in benchmark_data:
    bleu_score = compute_bleu(data["gold_response_tokens"], data["llm_response_tokens"])
    data["bleu_score"] = bleu_score

# # Print results
for data in benchmark_data:
    #print(f"Query: {data['query']}")
    #print(f"Gold Response: {data['gold_response']}")
    #print(f"LLM Response: {data['llm_response']}")
    print(f"BLEU Score: {data['bleu_score']:.4f}\n")


BLEU Score: 0.0233

BLEU Score: 0.0097

BLEU Score: 0.0128

BLEU Score: 0.0170

BLEU Score: 0.0117

BLEU Score: 0.0053

BLEU Score: 0.0132

BLEU Score: 0.0071

BLEU Score: 0.0039

BLEU Score: 0.0024

BLEU Score: 0.0045

BLEU Score: 0.0085

BLEU Score: 0.0023

BLEU Score: 0.0044

BLEU Score: 0.0236

BLEU Score: 0.0257

BLEU Score: 0.0045

BLEU Score: 0.0303

BLEU Score: 0.0054

BLEU Score: 0.0041

BLEU Score: 0.0053

BLEU Score: 0.0081

BLEU Score: 0.0137

BLEU Score: 0.0041

BLEU Score: 0.0111

BLEU Score: 0.0066

BLEU Score: 0.0049

BLEU Score: 0.0047

BLEU Score: 0.0083

BLEU Score: 0.0039

